
$$b'(s)=\frac{p(y|s)\cdot b(s)}{\int p(y|s)\cdot b(s)}$$

given $b(s)=\sum_i w_i \cdot \phi_i(s)$

$$b'(s)=\frac{p(y|s)\cdot \sum_i w_i \cdot \phi_i(s)}{\int p(y|s)\cdot \sum_i w_i \cdot \phi_i(s) ds} $$

imposing $b'(s)=\sum_i \tilde{w_i} \cdot \phi_i(s)$, then $\forall i$:


$$\tilde{w_i} = \frac{p(y|s)\cdot w_i}{\sum_i w_i\int p(y|s) \cdot \phi_i(s) ds} $$

$$\tilde{w_i} = \eta \cdot p(y|s)\cdot w_i $$

basta a uel punto normalizzare i wtilde così che int b ds = 1

computing $\int p(y|s) \cdot \phi_i(s) ds$ ???

$\#s = \#\theta \cdot \#pos $

affinché $\int b ds = 1$ è sufficiente che $\sum_i w_i = 1$ se le $\phi_i$ sono normalizzate, non serve calcolare l'integrale

Aspetta però potrei avere un altro problema....  p(y|s) dipende da s

ulteriore problema .. p(y|s) dovrebbe essere una distribuzione di probabilità dipendente da s, non da qualche f(s) che rende la trattazione esatta molto più complessa. Per come abbiamo strutturato il problema vale la seguente:
$$p(y|s) = p(\psi^{-1}(y,s)) = Deterministic 0 1 $$

 \approx \mathcal{N}(\psi^{-1}(y,s),1)


In [9]:
import numpy as np

class TileCoding:
    def __init__(self, dims, tile_widths, num_tilings):
        """
        Initialize Tile Coding parameters.

        Args:
            dims (int): Dimensionality of the input space.
            tile_widths (list or np.array): Width of each tile along each dimension.
            num_tilings (int): Number of overlapping tilings.
        """
        self.dims = dims
        self.tile_widths = np.array(tile_widths)
        self.num_tilings = num_tilings
        self.offsets = [np.random.uniform(0, tile_widths) for _ in range(num_tilings)]

    def get_active_tiles(self, s):
        """
        Get indices of active tiles for a given input vector.

        Args:
            s (np.array): Input vector of size `dims`.

        Returns:
            list: A list of active tile indices (one per tiling).
        """
        active_tiles = []

        for offset in self.offsets:
            shifted_s = (s + offset) / self.tile_widths
            indices = np.floor(shifted_s).astype(int)
            active_tiles.append(tuple(indices))

        return active_tiles

    def compute_basis_functions(self, s):
        """
        Compute the basis functions \( \phi_i(s) \).

        Args:
            s (np.array): Input vector of size `dims`.

        Returns:
            dict: A dictionary with active tile indices as keys and \( \phi_i(s) \) values.
        """
        active_tiles = self.get_active_tiles(s)
        basis_functions = {tile: 1 / self.num_tilings for tile in active_tiles}
        return basis_functions

class TileCodingFunction:
    def __init__(self, dims, tile_widths, num_tilings):
        """
        Initialize a function approximator using Tile Coding.

        Args:
            dims (int): Dimensionality of the input space.
            tile_widths (list or np.array): Width of each tile along each dimension.
            num_tilings (int): Number of overlapping tilings.
        """
        self.tile_coding = TileCoding(dims, tile_widths, num_tilings)
        self.weights = {}

    def set_weights(self, weights):
        """
        Set weights for the active tiles.

        Args:
            weights (dict): A dictionary mapping tile indices to weights.
        """
        self.weights = weights

    def evaluate(self, s):
        """
        Evaluate the function \( f(s) \) for a given input vector.

        Args:
            s (np.array): Input vector of size `dims`.

        Returns:
            float: The value of \( f(s) \).
        """
        basis_functions = self.tile_coding.compute_basis_functions(s)
        f_value = sum(self.weights.get(tile, 0) * phi for tile, phi in basis_functions.items())
        return f_value

# Example Usage
dims = 2  # Input space dimension
tile_widths = [1.0, 1.0]  # Width of each tile along each dimension
num_tilings = 5  # Number of overlapping tilings

# Initialize Tile Coding Function
function_approximator = TileCodingFunction(dims, tile_widths, num_tilings)

# Set some example weights
# example_weights = {
#     (0, 0, 0): 0.2,
#     (1, 1, 1, 1): 0.5,
#     (2, 2, 2, 2): 0.3,
# }
# function_approximator.set_weights(example_weights)

# Evaluate the function for an input vector s
s = np.array([0, 0,])
value = function_approximator.evaluate(s)
print(f"f(s) = {value}")


f(s) = 0.0
